In [ ]:
def inputify(fileName):
    return fileName
def outputify(fileName):
    return fileName

In [ ]:
# Imports go here
import sys
import warnings
import io
import os
from pathlib import Path

with warnings.catch_warnings():  
    warnings.filterwarnings("ignore", category=FutureWarning)
    import pandas as pd
    import numpy as np
    from keras.models import Sequential
    from keras.layers import Dense
    from keras.layers import Dropout
    from keras.layers import Flatten
    from keras.layers.convolutional import Conv2D
    from keras.layers.convolutional import MaxPooling2D
    from keras.utils import np_utils
    from sklearn import metrics
    from sklearn.model_selection import train_test_split
    
working_directory = str(Path.home()) + "/"
class_names = ['0','1','2','3','4','5','6','7','8','9','A','B','C','D','E','F','G','H','I','J','K','L',
'M','N','O','P','Q','R','S','T','U','V','W','X','Y','Z','a','b','d','e','f','g','h','n','q','r','t']
models = []

In [ ]:
def get_data():
    # Must use a read_csv call with file_path variable and output dataFrame
    inputName = inputify(working_directory + 'input/emnist_balanced.csv')
    data = pd.read_csv(inputName, header=None)
    return data

def prep_data(data):
    # Split first column off as label we're predicting
    X = data.iloc[:,1:]
    y = data.iloc[:,0]
    train_x, test_x, train_y, test_y = train_test_split(X, y, test_size=0.16, random_state=42)

    # Images need to be rotated before training
    train_x = np.asarray(train_x)
    test_x = np.asarray(test_x)
    train_x = np.apply_along_axis(rotate, 1, train_x)
    test_x = np.apply_along_axis(rotate, 1, test_x)

    # Normalization of grey-scale values
    train_x = train_x.astype('float32') / 255
    test_x = test_x.astype('float32') / 255

    # One-hot enconding of labels
    num_classes = y.nunique()
    train_y = np_utils.to_categorical(train_y, num_classes)
    test_y = np_utils.to_categorical(test_y, num_classes)

    # Reshaping to 28x28x1 image of grey-scale values for CNN
    train_x = train_x.reshape(train_x.shape[0], 28, 28, 1)
    test_x = test_x.reshape(test_x.shape[0], 28, 28, 1)

    return train_x, train_y, test_x, test_y, num_classes

def rotate(image):
    image = image.reshape([28, 28])
    image = np.fliplr(image)
    image = np.rot90(image)
    return image

# Use model_flag when naming files to be saved and transmit back at the end
data = get_data()

train_x, train_y, test_x, test_y, num_classes = prep_data(data)

In [ ]:
# Function can either return compiled model or fitted model if training data is passed as params
model = Sequential()
model.add(Flatten())
model.add(Dense(units=784, activation='relu'))
model.add(Dense(units=47, activation='softmax'))
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.fit(train_x, train_y, validation_data=(test_x, test_y), epochs=1)
models.append((1, model))

In [ ]:
model = Sequential()
model.add(Flatten())
model.add(Dense(units=784, activation='relu'))
model.add(Dense(units=168, activation='selu'))
model.add(Dense(units=84, activation='selu'))
model.add(Dense(units=47, activation='softmax'))
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.fit(train_x, train_y, validation_data=(test_x, test_y), epochs=1)
models.append((2, model))

In [ ]:
model = Sequential()
model.add(Conv2D(32, (4, 4), input_shape=(28, 28, 1), activation='relu'))
model.add(MaxPooling2D(pool_size=(3, 3)))
model.add(Dropout(0.2))
model.add(Conv2D(32, (3, 3), input_shape=(28, 28, 1), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.15))
model.add(Flatten())
model.add(Dense(units=188, activation='relu'))
model.add(Dropout(0.1))
model.add(Dense(units=47, activation='softmax'))
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.fit(train_x, train_y, validation_data=(test_x, test_y), epochs=1)
models.append((3, model))

In [ ]:
for i, model in models:
    jsonOutputName = outputify(working_directory + 'output/model' + str(i) + '.json')
    h5OutputName = outputify(working_directory + 'output/model' + str(i) + '.h5')
    model_json = model.to_json()
    with open(jsonOutputName, 'w') as json_file:
        json_file.write(model_json)
    model.save_weights(h5OutputName)